In [1]:
import numpy as np
import pandas as pd
import librosa #用來提取音頻特徵MFCC

import tensorflow as tf
from tensorflow.keras.layers import Activation, BatchNormalization, Dense, LayerNormalization
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, recall_score

In [3]:
# default hyperparameters
NEURONS = 300                    #每個隱藏層的神經元數量       
HIDDEN_LAYERS = 3                #隱藏層的層數 3層

def create_DNN(input_shape, neurons = NEURONS, hidden_layers = HIDDEN_LAYERS, learning_rate = 0.001, verbose=0):      #定義一個函數用於建立深度神經網路模型
    model = Sequential()                                                        # 定義一個Sequential物件作為模型容器

    model.add(Dense(neurons, input_dim=input_shape))                            # 設定神經元數目及輸入層維度
    model.add(Activation('relu'))                                               # 設定激活函數為ReLU

    for i in range(hidden_layers-1):                                            # 建立模型的隱藏層
        model.add(Dense(neurons))                                               # 設定神經元數目
        model.add(Activation('sigmoid'))                                        # 設定激活函數為sigmoid

    # 建立模型的輸出層
    model.add(Dense(3))                                                         # 設定神經元數目為3，代表輸出為三個類別：開心、中性、憂鬱
    model.add(Activation('softmax'))                                            # 設定激活函數為softmax

    opt = tf.keras.optimizers.Adam(learning_rate=learning_rate)                 # 設定優化器及損失函數
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy']) # categorical_crossentropy

    if verbose:                                                                 # 如果verbose為1，印出模型摘要
        model.summary()                                                         
    
    return model                                                                # 回傳建立好的模型

特徵訓練


In [ ]:
model = create_DNN(input_shape = x_train.shape[1], verbose=1)                   

In [ ]:
MODEL_SAVE_NAME = "AI_CUP_medical_sample_model"                                                             #模型名稱

train_results = model.fit(x_train, y_train, batch_size=None, epochs=100,                                    #訓練時每批次的數據量=32、整個數據集重複訓練的次數100
                              callbacks=[EarlyStopping(monitor='val_loss', patience=10, mode='auto'),       #EarlyStopping：訓練早期停止機制，當監測的指標停滯不前時提前停止訓練
                                         ModelCheckpoint(MODEL_SAVE_NAME+".h5", save_best_only=True)],      #ModelCheckpoint：定期保存模型參數，可以在訓練過程中保存最佳的模型參數
                              validation_data=(x_val, y_val))                                               #validation_data：驗證數據

data training

In [ ]:
y_pred = model.predict(training_df.iloc[:, :-1]).argmax(axis=1)                                         # 進行模型預測，取得每個樣本預測結果的類別編號
y_true = training_df['Disease category'] - 1                                                            # 取得每個樣本實際的類別編號

results_recall = recall_score(y_true, y_pred, average=None)                                             # 計算每個類別的召回率
print("Training UAR(Unweighted Average Recall) :", results_recall.mean())                               # 輸出平均召回率
ConfusionMatrixDisplay(confusion_matrix(y_true, y_pred)).plot(cmap='Blues')                             # 輸出混淆矩陣的圖表表示，以藍色調為主調

驗證資料Test result

In [ ]:
y_pred = model.predict(test_df.iloc[:, :-1]).argmax(axis=1)
y_true = test_df['Disease category'] - 1

results_recall = recall_score(y_true, y_pred, average=None)
print("Test UAR(Unweighted Average Recall) :", results_recall.mean())
ConfusionMatrixDisplay(confusion_matrix(y_true, y_pred)).plot(cmap='Blues')